<a href="https://colab.research.google.com/github/alunfes/GoogleColabProjects/blob/main/SynthesisArbAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import pandas as pd
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import multiprocessing as mp
import numpy as np
import itertools
import re

In [29]:
directory = '/content/drive/My Drive/MarketData/ohlc'

In [34]:


def adjust_ohlc_values(df, filename):
    """
    Adjusts the OHLC values in the dataframe based on the filename.

    Parameters:
    df (pandas.DataFrame): The dataframe containing the OHLC data.
    filename (str): The name of the file.

    Returns:
    pandas.DataFrame: The adjusted dataframe.
    """

    # Extract the multiplier from the filename using regex
    multiplier_match = re.search(r'(\d+)', filename)
    if multiplier_match:
        multiplier = int(multiplier_match.group(1))

        # Divide the OHLC values by the multiplier
        df['open'] /= multiplier
        df['high'] /= multiplier
        df['low'] /= multiplier
        df['close'] /= multiplier
    return df

def read_data():
    cleansed_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.startswith('cleansed_') and f.endswith('.csv')]
    filenames = []
    target_files = {}
    for f in cleansed_files:
        filenames.append(f.split('_')[1].split('.')[0])
        target_files[filenames[-1]] = adjust_ohlc_values(pd.read_csv(f), filenames[-1])
    return target_files

target_files = read_data()

In [52]:
# 各取引所のペアリストを取得
okx_pairs = [pair.replace('okx-', '') for pair in target_files.keys() if pair.startswith('okx-')]
bybit_pairs = [pair.replace('bybit-', '') for pair in target_files.keys() if pair.startswith('bybit-')]
apexpro_pairs = [pair.replace('apexpro-', '') for pair in target_files.keys() if pair.startswith('apexpro-')]

# 各取引所間で共通のペアを見つける
common_pairs = set(okx_pairs + bybit_pairs + apexpro_pairs)

# 結果を格納するためのリストを作成
results = []

# 各取引所間の価格差を計算し、統計情報を出力
for pair in common_pairs:
    exchanges = []
    if pair in okx_pairs:
        exchanges.append('okx')
    if pair in bybit_pairs:
        exchanges.append('bybit')
    if pair.replace('-USDT', '-USDC') in apexpro_pairs:
        exchanges.append('apexpro')

    # 少なくとも2つ以上の取引所でbase/quoteが一致している場合に価格差を比較
    if len(exchanges) >= 2:
        prices = []
        for exchange in exchanges:
            pair_name = f"{exchange}-{pair.replace('-USDT', '-USDC') if exchange == 'apexpro' else pair}"

            # apexproの価格をUSDT基準に修正
            if exchange == 'apexpro':
                target_files[pair_name]['close'] *= usdc_usdt

            prices.append(target_files[pair_name]['close'])

        # 各時刻における最大と最小の価格を計算
        max_price = pd.concat(prices, axis=1).max(axis=1)
        min_price = pd.concat(prices, axis=1).min(axis=1)

        # 価格差を計算
        price_diff = max_price - min_price

        # 価格差の統計情報をリストに追加
        results.append({
            'Pair': pair,
            'Exchanges': ', '.join(exchanges),
            'Max': price_diff.max(),
            'Min': price_diff.min(),
            'Mean': price_diff.mean(),
            'Std': price_diff.std()
        })

# 結果のリストをデータフレームに変換
result_df = pd.DataFrame(results)

# 結果のデータフレームを表示
result_df

,Pair,Exchanges,Max,Min,Mean,Std
0,JST-USDT,"okx, bybit",0.000180,0.00000,0.000017,0.000015
1,XRP-USDT,"okx, bybit, apexpro",0.015518,0.00000,0.000680,0.000643
2,GMX-USDT,"okx, bybit",0.190000,0.00000,0.025254,0.019608
3,FIL-USDT,"okx, bybit",0.049000,0.00000,0.001723,0.001301
4,ARB-USDT,"okx, bybit, apexpro",0.032897,0.00002,0.001903,0.001546
...,...,...,...,...,...,...
106,ENJ-USDT,"okx, bybit",0.001200,0.00000,0.000258,0.000190
107,CRO-USDT,"okx, bybit",0.000500,0.00000,0.000064,0.000052
108,ZEN-USDT,"okx, bybit",0.034000,0.00000,0.004704,0.003767
109,PEOPLE-USDT,"okx, bybit",0.000110,0.00000,0.000022,0.000015


In [ ]:
result_df['Exchanges']

In [43]:
common_pairs

set()

In [ ]:
okx_pairs